### WRITING A TWEET COLLECTOR IN PYTHON

In [1]:
from twython import Twython
import csv
import os
from pandas import DataFrame, Series
import pandas as pd
import numpy as np

In [4]:
# Authorization
APP_KEY = "2lzEpNDwz9ieHhr738sXX8Avl"
APP_SECRET = "z3srlvggHvhCqIBSMnKRbn5OY36B2Z58299ipfKtuRXMqwI4rj"

twitter_get_token = Twython(APP_KEY,APP_SECRET,oauth_version=2)
ACCESS_TOKEN = twitter_get_token.obtain_access_token()

twitter = Twython(APP_KEY,access_token = ACCESS_TOKEN)

In [32]:
def crawl_tweets(topic_string,num_of_calls):
    """A function to crawl tweets
       (1) query topic is specified by "topic_string"
       (2) num_of_calls has an upper limit of 450 per access token which is valid for 15 min;
           each call will crawl 100 tweets
       (3) returns a list, each element contains 100 tweets"""
    empty_list = []
    for i in range(num_of_calls):
        empty_list.append(twitter.search(q=topic_string,lang = 'en',count=100))
    return empty_list

In [35]:
def one_tweet_per_element(raw_tweets_list):
    empty_list = []
    for i in range(len(raw_tweets_list)):
        empty_list += raw_tweets_list[i]['statuses']
    return empty_list

In [36]:
raw_tweets = crawl_tweets("zika",100)

In [63]:
messaged_tweets = one_tweet_per_element(raw_tweets)

In [61]:
len(messaged_tweets)

10000

In [75]:
# formatting to nice csv-ish file
def collect_tweet_level_field_from_each_user(title_string,messaged_tweets_list):
    """Retrieve info at per-tweet level"""
    empty_list = []
    for i in range(len(messaged_tweets_list)):
        empty_list.append(messaged_tweets_list[i][title_string])
    return empty_list

In [76]:
def collect_user_level_field_from_each_user(title_string,messaged_tweets_list):
    """Retrieve info at per-user level"""    
    empty_list = []
    for i in range(len(messaged_tweets_list)):
        empty_list.append(messaged_tweets_list[i]['user'][title_string])
    return empty_list

In [77]:
# Tweet level fields
messaged_tweets[0].keys()

dict_keys(['entities', 'in_reply_to_user_id_str', 'source', 'place', 'retweeted', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'truncated', 'id', 'user', 'favorite_count', 'coordinates', 'favorited', 'created_at', 'text', 'possibly_sensitive', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'lang', 'geo', 'is_quote_status', 'retweet_count', 'id_str', 'contributors', 'metadata'])

In [78]:
tweet_id = collect_tweet_level_field_from_each_user('id',messaged_tweets)
text = collect_tweet_level_field_from_each_user('text',messaged_tweets)
retweet_count = collect_tweet_level_field_from_each_user('retweet_count',messaged_tweets)

In [79]:
# User level fields
messaged_tweets[0]['user'].keys()

dict_keys(['profile_banner_url', 'follow_request_sent', 'profile_sidebar_border_color', 'followers_count', 'entities', 'has_extended_profile', 'profile_background_image_url', 'description', 'profile_image_url', 'statuses_count', 'screen_name', 'is_translation_enabled', 'time_zone', 'is_translator', 'following', 'favourites_count', 'id', 'default_profile_image', 'profile_background_image_url_https', 'verified', 'profile_sidebar_fill_color', 'profile_text_color', 'utc_offset', 'profile_use_background_image', 'created_at', 'location', 'name', 'profile_link_color', 'profile_image_url_https', 'contributors_enabled', 'lang', 'friends_count', 'protected', 'geo_enabled', 'profile_background_color', 'default_profile', 'listed_count', 'id_str', 'url', 'profile_background_tile', 'notifications'])

In [80]:
user_id = collect_user_level_field_from_each_user('id',messaged_tweets)
screen_name = collect_user_level_field_from_each_user('screen_name',messaged_tweets)
following = collect_user_level_field_from_each_user('friends_count',messaged_tweets)
followers_count = collect_user_level_field_from_each_user('followers_count',messaged_tweets)
listed_count = collect_user_level_field_from_each_user('listed_count', messaged_tweets)
friends_count = collect_user_level_field_from_each_user('friends_count',messaged_tweets)
favourites_count =   collect_user_level_field_from_each_user('favourites_count',messaged_tweets)
statuses_count = collect_user_level_field_from_each_user('statuses_count', messaged_tweets)
location = collect_user_level_field_from_each_user('location',messaged_tweets)
created_at = collect_user_level_field_from_each_user('created_at',messaged_tweets)
verified = collect_user_level_field_from_each_user('verified',messaged_tweets)
description = collect_user_level_field_from_each_user('description',messaged_tweets)

In [89]:
tweets_dict = {
    'tweet_id' : tweet_id,
    'text' : text,
    'retweet_count' : retweet_count,
    'user_id' : user_id,
    'screen_name' : screen_name,
    'following' : following,
    'followers_count' : followers_count,
    'listed_count' : listed_count,
    'friends_count' : friends_count,
    'favourites_count' : favourites_count,
    'statuses_count' : statuses_count,
    'location' : location,
    'created_at' : created_at,
    'verified' : verified,
    'description' :description
}

In [90]:
tweets_df = pd.DataFrame(data = tweets_dict, columns = tweets_dict.keys())
tweets_df.head(5)

,followers_count,text,location,description,statuses_count,screen_name,following,favourites_count,friends_count,created_at,retweet_count,listed_count,user_id,verified,tweet_id
0,1596,A doctor connected to Putnam Hospital Center s...,"New City, NY",New City Patch is your source for local news.,12248,NewCityPatch,51,7,51,Tue Apr 06 18:05:54 +0000 2010,0,54,130226736,False,694206616946765824
1,1826,A doctor connected to Putnam Hospital Center s...,"Nyack & Piermont, NY",Nyack-Piermont Patch is your source for local ...,13669,NyackPatch,55,0,55,Tue Jun 22 19:44:23 +0000 2010,0,41,158467267,False,694206615101247488
2,714,#FollowForFollow WHO meets to decide if Zika v...,Kenya,Mention me after you tap that follow button fo...,79137,_Dirty_Water_,114,10,114,Thu Jul 19 21:04:22 +0000 2012,0,35,705784747,False,694206612680998913
3,2033,A doctor connected to Putnam Hospital Center s...,,Local news and conversation from Hudson Valley...,6231,HVPatch,871,41,871,Tue Jul 10 18:33:10 +0000 2012,0,46,632210824,False,694206612601491461
4,2553,RT @CNN: The #Zika virus could complicate the ...,En pèlerinage sur terre.,Je revendique le droit de donner mon avis sur ...,63435,PierreChrist_,691,9419,691,Fri Mar 06 13:15:41 +0000 2009,41,66,23061341,False,694206609422172161


In [91]:
tweets_df.shape

(10000, 15)

In [93]:
#exporting to csv
tweets_df.to_csv('zika_tweets_per_tweet_2.csv', encoding = 'utf-8')

### TASK A

In [94]:
zika = pd.read_csv('zika_tweets_per_tweet_2.csv')
zika.shape

(10000, 16)

In [100]:
#selecting subset of variables identified in HW1
taska = zika[['user_id','listed_count','retweet_count','followers_count','following']]
taska.shape

(10000, 5)

In [120]:
retweet_by_user = pd.DataFrame(taska.groupby('user_id')['retweet_count'].sum())
user_info = taska.groupby('user_id')[['listed_count','followers_count','following']].mean()
taska_by_user = pd.merge(user_info,retweet_by_user,left_index = True, right_index = True)

In [107]:
# Log-transform all features
def log_trans(x):
    return np.log(x+1)  # +1 to take care of 0 values

In [109]:
for column in taska_by_user:
    taska_by_user[column] = log_trans(taska_by_user[column])

In [112]:
#Coefficients learnt from HW1:
follower = 0.10919318
following = -0.00270281
listed = 0.2451884
retweet = 0.04542407

In [114]:
# Rescale s.t. sum up to 1
scaler = 1/(follower+following+listed+retweet)
follower *= scaler
following *= scaler
listed *= scaler
retweet *= scaler

In [21]:
taska_by_user['influence_score'] = taska_by_user['listed_count'] * listed + taska_by_user['followers_count'] * follower + taska_by_user['following'] * following + taska_by_user['retweet_count'] * retweet

# Rank by influence_score and get TOP 50
Influencer_top50 = taska_by_user.sort_values(by = 'influence_score', ascending = False)[:50]
Influencer_top50.head(10)

,listed_count,followers_count,following,retweet_count,influence_score
user_id,,,,,
27818124,7.403061,9.404508,7.522941,5.111988,7.690519
40954616,7.035269,12.012518,5.953243,0.000000,7.606495
149666064,5.948035,11.138770,7.611348,2.772589,7.000799
16469116,6.542472,10.590264,7.385851,0.000000,6.901387
49069711,5.877736,8.769196,7.126891,7.885329,6.893958
101806923,6.428105,8.725994,8.819813,4.615121,6.836303
2759000172,6.490724,10.095099,9.837989,0.000000,6.716587
189663569,5.808142,8.532476,8.285765,6.841615,6.658619
101034530,6.612041,9.242130,8.666992,0.000000,6.564919


Questions:
1. The coefficients are learnt on log(diff), can we directly apply that in the absolute case?
2. Why do we have to normalize features, given that coefficients are learnt from non-normalized features? 

### TASK B

In [122]:
import fnmatch
import nltk.data
from nltk.tokenize import RegexpTokenizer

In [125]:
zika = pd.read_csv("zika_tweets_per_tweet_2.csv", encoding='utf8')
def WordProcess(series):
    """This function will tokenize each row in a given series and return a list"""
    tokenized = []
    tokenizer = RegexpTokenizer(r'\w+')
    for i in range(len(series)):
        tokenized.append(tokenizer.tokenize(series[i]))
    return tokenized

In [126]:
#random sample 5000 tweets from the 9717 tweets we scraped
zika_5000 = zika.sample(n=5000, random_state = 123)
zika_df = zika_5000.reset_index(drop=True)
tweet = zika_df['text']
word_list = WordProcess(tweet) #tokenized each tweet

In [135]:
#create an empty dataframe with 3 columns and 5000 rows
columns = ['Column1', 'Column2', 'Column3']
df = pd.DataFrame(index = range(len(word_list)), columns=columns)

In [136]:
#fill in the dataframe with retweets
for index, tweet in enumerate(word_list):
    for ind, word in enumerate(tweet):
        if word =='RT': #if RT is in the tweet, the original tweet is word behind RT
            df.ix[index]['Column2'] = '@'+tweet[ind+1]
            df.ix[index]['Column3'] = word
            df.ix[index]['Column1'] = '@'+zika_df['screen_name'][index]
df = df.fillna('none')
for i in range(len(df)):
    #if there are no RT, then it's an original tweet
    if df['Column1'][i] == 'none':
        df.ix[i]['Column1'] = '@'+zika_df['screen_name'][i]
        df.ix[i]['Column2'] = '@'+zika_df['screen_name'][i]
        df.ix[i]['Column3'] = 'Tweet'

In [137]:
df.to_csv('task_B.csv', encoding = 'utf-8')